In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Exibir todas as linhas
pd.set_option('display.max_rows', None)

# Exibir todas as colunas
pd.set_option('display.max_columns', None)

In [ ]:
X = pd.DataFrame(pd.read_csv('files/playground-series-s4e6/train.csv'))
X.columns



In [ ]:
X.head()

In [ ]:
X['Target'].unique()

In [ ]:
# Criar um dicionário de mapeamento
target_map = {
    'Graduate': 1,
    'Dropout': 2,
    'Enrolled': 3
}

X['Ind_Target'] = X['Target'].map(target_map)

X.head()

In [ ]:
y = X[['Ind_Target']]

In [ ]:
y.head()

In [ ]:
X.drop('Target',axis=1,inplace=True)
X.head()


numerical_features = ['Previous qualification (grade)',"Admission grade","Displaced","Educational special needs","Debtor"
 ,"Gender","Tuition fees up to date","Scholarship holder","International"
 ,"Curricular units 1st sem (credited)", "Curricular units 1st sem (enrolled)"
 ,"Curricular units 1st sem (evaluations)", "Curricular units 1st sem (approved)"
 ,"Curricular units 1st sem (grade)", "Curricular units 1st sem (without evaluations)"
 ,"Curricular units 2nd sem (credited)", "Curricular units 2nd sem (enrolled)"
 ,"Curricular units 2nd sem (evaluations)", "Curricular units 2nd sem (approved)"
 ,"Curricular units 2nd sem (grade)", "Curricular units 2nd sem (without evaluations)"
 ,"Unemployment rate", "Inflation rate","GDP"]

categorical_features = ['Marital status'
 ,'Application mode'
 ,'Application order'
 ,'Course'
 ,'Previous qualification'
 ,'Nacionality'
 ,"Mother's qualification"
 ,"Father's qualification"
 ,"Mother's occupation"
 ,"Father's occupation"
 ,"Age at enrollment"
 ]

print(numerical_features)
print(categorical_features)


In [ ]:

# Normalização
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Amostragem
smote = SMOTE()
X, y = smote.fit_resample(X, y)

# Adicionar características polinomiais
poly = PolynomialFeatures(degree=2)
X = poly.fit_transform(X)

# Ajuste de hiperparâmetros
param_grid = {
    'C': [0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear']
}


In [ ]:
pd.DataFrame(X).head()

In [ ]:
# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [ ]:

grid = GridSearchCV(LogisticRegression(), param_grid, cv=5, scoring='accuracy')
grid.fit(X_train, y_train)


In [ ]:
grid.best_estimator_

In [ ]:
# Treinar o modelo com os melhores parâmetros
model = grid.best_estimator_
model.fit(X_train, y_train)


# Avaliação
y_pred = model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

print(f"Melhor acurácia: {grid.best_score_}")


In [ ]:
import joblib

joblib.dump(model, 'model_logistic.joblib')

In [ ]:
# Carregar o modelo salvo
model = joblib.load('model_logistic.joblib')

# Exemplo de novos dados para previsão
df_test = pd.read_csv('files/playground-series-s4e6/test.csv')

df_test.head()

In [ ]:

# Fazer previsões com o modelo carregado
predict = model.predict(df_test)

predict